In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import json
# A/B test
import tarfile
import requests
from io import BytesIO

import pandas as pd
import requests
import tarfile
from io import BytesIO

# Consumidores
url_consumers = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz"
df_consumers = pd.read_csv(url_consumers)
print(f"df_consumers shape: {df_consumers.shape}")


df_consumers shape: (806156, 7)


In [3]:
# Restaurantes
url_restaurants = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz"
df_restaurants = pd.read_csv(url_restaurants)
print(f"df_restaurants shape: {df_restaurants.shape}")


df_restaurants shape: (7292, 12)


In [4]:
# Teste A/B
url_ab = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz"
response = requests.get(url_ab)
tar = tarfile.open(fileobj=BytesIO(response.content))
tar.extractall()
tar.close()
df_ab = pd.read_csv("ab_test_ref.csv")
print(f"Dados A/B carregados: {df_ab.shape}")

Dados A/B carregados: (806467, 2)


In [5]:
# Pedidos (em chunk porque estava excedendo a memória RAM)
url_orders = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz"
chunksize = 100000

lista_agg = []

for chunk in pd.read_json(url_orders, lines=True, chunksize=chunksize):
    cols = ['customer_id', 'order_id', 'order_total_amount', 'order_created_at']
    df = chunk[cols]
    agg = df.groupby('customer_id').agg(
        num_pedidos=('order_id', 'count'),
        gasto_total=('order_total_amount', 'sum')
    ).reset_index()
    lista_agg.append(agg)

df_orders_agg = pd.concat(lista_agg)
df_orders_agg = df_orders_agg.groupby('customer_id').agg(
    num_pedidos=('num_pedidos', 'sum'),
    gasto_total=('gasto_total', 'sum')
).reset_index()
print(f"df_orders_agg shape: {df_orders_agg.shape}")

# 7min

df_orders_agg shape: (806466, 3)


In [6]:
df_ab = df_ab.rename(columns={'is_target': 'grupo'})

# Join pedidos + teste a/b (obs: existe 1 cliente que não fez pedido no período, mas esta na tabela a/b)
df_base = df_ab.merge(df_orders_agg, on='customer_id', how='left')

# Join com consumidores
df_base = df_base.merge(df_consumers[['customer_id', 'active']], on='customer_id', how='left')

# Tratar valores nulos na base final (clientes sem pedido e clientes que não estão na base de consumidores)
df_base['num_pedidos'] = df_base['num_pedidos'].fillna(0).astype(int)
df_base['gasto_total'] = df_base['gasto_total'].fillna(0)
df_base['active'] = df_base['active'].fillna(False)

df_base = df_base.rename(columns={'active': 'cliente_ativo'})

# Calculo do ticket médio por cliente (obs: temos o ticket médio por restaurante e por isso é necessário calcular por usuário)
df_base['ticket_medio'] = df_base.apply(
    lambda row: row['gasto_total'] / row['num_pedidos'] if row['num_pedidos'] > 0 else np.nan,
    axis=1
)

# Identificando se o cliente fez pedido no período
df_base['fez_pedido'] = df_base['num_pedidos'] > 0

df_base.head()


/tmp/ipython-input-6-4041337541.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_base['active'] = df_base['active'].fillna(False)


,customer_id,grupo,num_pedidos,gasto_total,cliente_ativo,ticket_medio,fez_pedido
0,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,target,19,229.00,True,12.052632,True
1,b821aa8372b8e5b82cdc283742757df8c45eecdd72adf4...,control,6,287.88,True,47.980000,True
2,d425d6ee4c9d4e211b71da8fc60bf6c5336b2ea9af9cc0...,control,31,2004.59,True,64.664194,True
3,6a7089eea0a5dc294fbccd4fa24d0d84a90c1cc12e829c...,target,10,445.43,True,44.543000,True
4,dad6b7e222bab31c0332b0ccd9fa5dbd147008facd268f...,control,1,23.00,True,23.000000,True


In [7]:
print("Total de usuários na base:", len(df_base))
print("\nDistribuição por grupo:")
print(df_base['grupo'].value_counts())

print("\nTicket médio (média dos usuários com pedidos):")
print(df_base[df_base['fez_pedido']].groupby('grupo')['ticket_medio'].mean())


Total de usuários na base: 806467

Distribuição por grupo:
grupo
target     445925
control    360542
Name: count, dtype: int64

Ticket médio (média dos usuários com pedidos):
grupo
control    48.011382
target     47.833608
Name: ticket_medio, dtype: float64


In [ ]:
df_base.to_csv('/content/drive/MyDrive/Case Ifood/df_base_tratada.csv', index=False)